In [4]:
# Import Necessary Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
# Device Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device", device)

Using device cuda


In [6]:
# Load MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=64,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=1000,
    shuffle=False
)

In [7]:
# Define MLP Model
class MLP(nn.Module):
    def __init__(self, use_dropout=False):
        super().__init__()

        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        if self.use_dropout:
            x = self.dropout(x)

        x = torch.relu(self.fc2(x))
        if self.use_dropout:
            x = self.dropout(x)

        x = self.fc3(x)
        return x

In [8]:
# Training Function
def train_model(model, optimizer, epochs=5):
    loss_fn = nn.CrossEntropyLoss()
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            preds = model(images)
            loss = loss_fn(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(preds, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        acc = correct / total
        print("Epoch", epoch, "Loss", total_loss, "Accuracy", acc)

In [9]:
# Test Function
def test_model(model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            preds = model(images)
            _, predicted = torch.max(preds, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print("Test Accuracy", correct / total)

In [10]:
# SGD Without Dropout
print("SGD without dropout")

model = MLP(use_dropout=False)
optimizer = optim.SGD(model.parameters(), lr=0.1)

train_model(model, optimizer)
test_model(model)

SGD without dropout
Epoch 0 Loss 262.3500819578767 Accuracy 0.9167166666666666
Epoch 1 Loss 97.53684905264527 Accuracy 0.9681666666666666
Epoch 2 Loss 64.64597795857117 Accuracy 0.9791833333333333
Epoch 3 Loss 45.78965648752637 Accuracy 0.98445
Epoch 4 Loss 34.44534702715464 Accuracy 0.9888833333333333
Test Accuracy 0.979


In [11]:
# SGD with Momentum
print("SGD with momentum")

model = MLP(use_dropout=False)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

train_model(model, optimizer)
test_model(model)

SGD with momentum
Epoch 0 Loss 288.1163061633706 Accuracy 0.9122
Epoch 1 Loss 183.64383629523218 Accuracy 0.94945
Epoch 2 Loss 149.28061651461758 Accuracy 0.9593166666666667
Epoch 3 Loss 141.38925559923518 Accuracy 0.96255
Epoch 4 Loss 143.08519721234916 Accuracy 0.9641833333333333
Test Accuracy 0.9531


In [12]:
# Adam Without Dropout
print("Adam without dropout")

model = MLP(use_dropout=False)
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, optimizer)
test_model(model)

Adam without dropout
Epoch 0 Loss 213.9676081314683 Accuracy 0.9311166666666667
Epoch 1 Loss 85.09621845930815 Accuracy 0.9713333333333334
Epoch 2 Loss 60.22947509912774 Accuracy 0.9790666666666666
Epoch 3 Loss 45.56835734983906 Accuracy 0.9841666666666666
Epoch 4 Loss 35.60322344134329 Accuracy 0.9877666666666667
Test Accuracy 0.9705


In [13]:
# Adam With Dropout
print("Adam with dropout")

model = MLP(use_dropout=True)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

train_model(model, optimizer)
test_model(model)

Adam with dropout
Epoch 0 Loss 386.2008899971843 Accuracy 0.8745333333333334
Epoch 1 Loss 213.05356270074844 Accuracy 0.93455
Epoch 2 Loss 182.4619315341115 Accuracy 0.9428666666666666
Epoch 3 Loss 164.17205951549113 Accuracy 0.9483
Epoch 4 Loss 152.6263959519565 Accuracy 0.9524333333333334
Test Accuracy 0.9714
